In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [2]:
import keras_tuner as kt
from keras_tuner import RandomSearch
# from GEH_LOSS import GEH_loss
import geh as ga

In [3]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [4]:
import relative_accuracy as ra

In [5]:
# Load all Inputs and Output Data

# 5 mins (1 step ahead)
Deep_train_5   = np.load("Deep_train_5_multi.npz")['x'] 
Output_train_5 = np.load("Deep_train_5_multi.npz")['y']

Deep_test_5   = np.load("Deep_test_5_multi.npz")['x'] 
Output_test_5 = np.load("Deep_test_5_multi.npz")['y'] 

# 15 mins (3 steps ahead)
Deep_train_15   = np.load("Deep_train_15_multi.npz")['x'] 
Output_train_15 = np.load("Deep_train_15_multi.npz")['y']

Deep_test_15   = np.load("Deep_test_15_multi.npz")['x'] 
Output_test_15 = np.load("Deep_test_15_multi.npz")['y']

# 30 mins (6 steps ahead)
Deep_train_30   = np.load("Deep_train_30_multi.npz")['x'] 
Output_train_30 = np.load("Deep_train_30_multi.npz")['y']

Deep_test_30   = np.load("Deep_test_30_multi.npz")['x'] 
Output_test_30 = np.load("Deep_test_30_multi.npz")['y']

# 60 mins (12 steps ahead)
Deep_train_60   = np.load("Deep_train_60_multi.npz")['x'] 
Output_train_60 = np.load("Deep_train_60_multi.npz")['y']

Deep_test_60   = np.load("Deep_test_60_multi.npz")['x'] 
Output_test_60 = np.load("Deep_test_60_multi.npz")['y']

In [6]:
# Wide Data
Xtrain = joblib.load("002weeks_train_Multi.save") 
Xtest = joblib.load("002weeks_test_Multi.save") 

In [7]:
# 5 mins
# Delete first 15 samples
Wide_train_5 = np.delete(Xtrain, np.s_[0:15], 0)
Wide_test_5 = np.delete(Xtest, np.s_[0:15], 0)

In [8]:
# 15 mins
# Delete first 17 samples
Wide_train_15 = np.delete(Xtrain, np.s_[0:17], 0)
Wide_test_15 = np.delete(Xtest, np.s_[0:17], 0)

In [9]:
# 30 mins
# Delete first 20 samples
Wide_train_30 = np.delete(Xtrain, np.s_[0:20], 0)
Wide_test_30 = np.delete(Xtest, np.s_[0:20], 0)

In [10]:
# 60 mins
# Delete first 27 samples
Wide_train_60 = np.delete(Xtrain, np.s_[0:26], 0)
Wide_test_60 = np.delete(Xtest, np.s_[0:26], 0)

In [11]:
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename) 

In [12]:
Test      = pd.read_csv('01test_scaled.csv', index_col=None, parse_dates=[0])

In [13]:
from pyts.image import GramianAngularField

# Reshape Samples

The first step is to split the input sequences into subsequences that can be processed by the CNN model. Here, each spatio-temporal sample can be split into three sub-samples, each with five time steps. The CNN can interpret each subsequence of five time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [14]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 6                # No of loop detectors
n_seq = 3                     # Subsequences
n_steps = 5                   # time-step per subsequence
# val_percent = 0.07567       # 2 weeks
val_percent = 0.2        
batch_size = 32
n_GAF = 15

n_seq_ = 3                     # Subsequences
n_steps_ = 3                   # time-step per subsequence

In [15]:
# Summation
gasf = GramianAngularField(image_size=1., method='summation', sample_range=None)
# Difference
gadf = GramianAngularField(image_size=1., method='difference', sample_range=None)

#### 5 mins

In [16]:
# Train
Xt5 = np.empty((len(Deep_train_5), n_features, n_GAF, n_GAF))
yt5 = scaler.inverse_transform(Output_train_5)

In [17]:
# Test
Xv5 = np.empty((len(Deep_test_5), n_features, n_GAF, n_GAF))
yv5 = scaler.inverse_transform(Output_test_5)

In [18]:
Deep_train_5 = np.where(np.isclose(Deep_train_5, 1, rtol=0, atol=1e-9), 1, Deep_train_5)
Deep_test_5 = np.where(np.isclose(Deep_test_5, 1, rtol=0, atol=1e-9), 1, Deep_test_5)           

#### 15 mins

In [19]:
# Train
Xt15 = np.empty((len(Deep_train_15), n_features, n_GAF, n_GAF))
yt15 = scaler.inverse_transform(Output_train_15)

In [20]:
# Test
Xv15 = np.empty((len(Deep_test_15), n_features, n_GAF, n_GAF))
yv15 = scaler.inverse_transform(Output_test_15)

In [21]:
Deep_train_15 = np.where(np.isclose(Deep_train_15, 1, rtol=0, atol=1e-9), 1, Deep_train_15)
Deep_test_15 = np.where(np.isclose(Deep_test_15, 1, rtol=0, atol=1e-9), 1, Deep_test_15)     

#### 30 mins

In [22]:
# Train
Xt30 = np.empty((len(Deep_train_30), n_features, n_GAF, n_GAF))
yt30 = scaler.inverse_transform(Output_train_30)

In [23]:
# Test
Xv30 = np.empty((len(Deep_test_30), n_features, n_GAF, n_GAF))
yv30 = scaler.inverse_transform(Output_test_30)

In [24]:
Deep_train_30 = np.where(np.isclose(Deep_train_30, 1, rtol=0, atol=1e-9), 1, Deep_train_30)
Deep_test_30 = np.where(np.isclose(Deep_test_30, 1, rtol=0, atol=1e-9), 1, Deep_test_30)    

#### 60 mins

In [25]:
# Train
Xt60 = np.empty((len(Deep_train_60), n_features, n_GAF, n_GAF))
yt60 = scaler.inverse_transform(Output_train_60)

In [26]:
# Test
Xv60 = np.empty((len(Deep_test_60), n_features, n_GAF, n_GAF))
yv60 = scaler.inverse_transform(Output_test_60)

In [27]:
Deep_train_60 = np.where(np.isclose(Deep_train_60, 1, rtol=0, atol=1e-9), 1, Deep_train_60)
Deep_test_60 = np.where(np.isclose(Deep_test_60, 1, rtol=0, atol=1e-9), 1, Deep_test_60) 

In [28]:
## Check Size

In [29]:
Xt5.shape

(49233, 6, 15, 15)

## Tranformations

### 5 mins

In [30]:
for i in range(len(Xt5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_5[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt5[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [31]:
for i in range(len(Xv5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_5[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv5[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [32]:
Xt5.shape

(49233, 6, 15, 15)

In [33]:
Xv5.shape

(4017, 6, 15, 15)

### 15 mins

In [34]:
for i in range(len(Xt15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_15[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt15[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [35]:
for i in range(len(Xv15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_15[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv15[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [36]:
Xt15.shape

(49231, 6, 15, 15)

In [37]:
Xv15.shape

(4015, 6, 15, 15)

### 30 mins

In [38]:
for i in range(len(Xt30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_30[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt30[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [39]:
for i in range(len(Xv30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_30[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv30[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [40]:
Xt30.shape

(49228, 6, 15, 15)

In [41]:
Xv30.shape

(4012, 6, 15, 15)

### 60 mins

In [42]:
for i in range(len(Xt60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_60[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt30[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [43]:
for i in range(len(Xv60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_60[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv60[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [44]:
Xt60.shape

(49222, 6, 15, 15)

In [45]:
Xv60.shape

(4006, 6, 15, 15)

### Reshaping the Data for Conv2D layer

For Conv2D, there is a need to add one more dimension to show we're dealing with 1 channel (since technically the images are in black and white, only showing values from 0-max flow on a single channel).

Conv1D - strides in 1 dimension
Conv2D - strides in 2 dimensions

In [46]:
# Train data
Xt5  =  np.transpose(Xt5, (0, 2, 3, 1))
Xt15 =  np.transpose(Xt15, (0, 2, 3, 1))
Xt30 =  np.transpose(Xt30, (0, 2, 3, 1))
Xt60 =  np.transpose(Xt60, (0, 2, 3, 1))

# Validation data
Xv5  =  np.transpose(Xv5, (0, 2, 3, 1))
Xv15 =  np.transpose(Xv15, (0, 2, 3, 1))
Xv30 =  np.transpose(Xv30, (0, 2, 3, 1))
Xv60 =  np.transpose(Xv60, (0, 2, 3, 1))

In [47]:
Xt5.shape

(49233, 15, 15, 6)

In [48]:
Xv5.shape

(4017, 15, 15, 6)

In [49]:
# Train data
Xt5  =  Xt5.reshape(Xt5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt15 =  Xt15.reshape(Xt15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt30 =  Xt30.reshape(Xt30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt60 =  Xt60.reshape(Xt60.shape[0], n_seq, n_steps, n_GAF, n_features)

# Validation data
Xv5  =  Xv5.reshape(Xv5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv15 =  Xv15.reshape(Xv15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv30 =  Xv30.reshape(Xv30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv60 =  Xv60.reshape(Xv60.shape[0], n_seq, n_steps, n_GAF, n_features)

In [50]:
Wt5  = Wide_train_5
Wt15 = Wide_train_15
Wt30 = Wide_train_30
Wt60 = Wide_train_60

Wv5  = Wide_test_5
Wv15 = Wide_test_15
Wv30 = Wide_test_30
Wv60 = Wide_test_60

In [51]:
# Train data
Wt5  =  Wt5.reshape(Wt5.shape[0], n_seq_, n_steps_, n_features)
Wt15 =  Wt15.reshape(Wt15.shape[0], n_seq_, n_steps_, n_features)
Wt30 =  Wt30.reshape(Wt30.shape[0], n_seq_, n_steps_, n_features)
Wt60 =  Wt60.reshape(Wt60.shape[0], n_seq_, n_steps_, n_features)

# Validation data
Wv5  =  Wv5.reshape(Wv5.shape[0], n_seq_, n_steps_, n_features)
Wv15 =  Wv15.reshape(Wv15.shape[0], n_seq_, n_steps_, n_features)
Wv30 =  Wv30.reshape(Wv30.shape[0], n_seq_, n_steps_, n_features)
Wv60 =  Wv60.reshape(Wv60.shape[0], n_seq_, n_steps_, n_features)

In [52]:
Xt5.shape

(49233, 3, 5, 15, 6)

In [53]:
Wt5.shape

(49233, 3, 3, 6)

# CNN-LSTM model

In [54]:
from tensorflow import keras
from keras_self_attention import SeqSelfAttention

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights = True)

In [55]:
Wide_train_5.shape

(49233, 9, 6)

# 5 mins ahead

In [56]:
def build_model_5(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt5.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt5.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_5w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_5 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_5w)
    
    
    # Deep Model
    # CNN 
    CNN_5 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_5 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_5)
    # LSTM          
    LSTM_5 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_5)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_5, LSTM_5], axis=1)

    # Attention
    Att_5 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)
    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    #print(output.shape)
    
    # Model
    model_5 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_5.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                                   
    return model_5

In [57]:
tuner5 = RandomSearch(
    build_model_5,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10a-5",
)

In [58]:
tuner5.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [59]:
tuner5.search((Wt5,Xt5), yt5, epochs=200,
           validation_split = val_percent,
           callbacks = [early_stop],
           verbose=2)

Trial 10 Complete [00h 23m 02s]
val_loss: 727.9768676757812

Best val_loss So Far: 402.4665069580078
Total elapsed time: 10h 45m 17s
INFO:tensorflow:Oracle triggered exit


In [60]:
tuner5.results_summary(1)

Results summary
Results in C:\RunsOak\10a-5
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 05 summary
Hyperparameters:
units: 80
filters: 480
kernel_size: 2
learning_rate: 0.0001
Score: 402.4665069580078


### Best Model

In [61]:
best_hp5 = tuner5.get_best_hyperparameters()[0]
model5 = tuner5.hypermodel.build(best_hp5)

In [62]:
scaler_filename = "Best_HP/10a-5"
joblib.dump(best_hp5, scaler_filename) 

['Best_HP/10a-5']

In [63]:
scaler_filename = "Best_HP/10a-5"
best_hp5 = joblib.load(scaler_filename) 

model5 = tuner5.hypermodel.build(best_hp5)

In [64]:
model5.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 480 12000       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [65]:
model5.fit((Wt5,Xt5), yt5, validation_split = val_percent,
           epochs = 100, 
           verbose = 1)

Epoch 1/100
1231/1231 [==============================] - 73s 56ms/step - loss: 23638.3887 - mean_absolute_error: 119.7864 - root_mean_squared_error: 153.7479 - mean_absolute_percentage_error: 89.8638 - val_loss: 18476.4824 - val_mean_absolute_error: 102.3614 - val_root_mean_squared_error: 135.9283 - val_mean_absolute_percentage_error: 105.4675
Epoch 2/100
1231/1231 [==============================] - 69s 56ms/step - loss: 13971.8799 - mean_absolute_error: 87.9010 - root_mean_squared_error: 118.2028 - mean_absolute_percentage_error: 114.1693 - val_loss: 11447.9385 - val_mean_absolute_error: 77.2110 - val_root_mean_squared_error: 106.9951 - val_mean_absolute_percentage_error: 110.8341
Epoch 3/100
1231/1231 [==============================] - 71s 57ms/step - loss: 8580.7549 - mean_absolute_error: 65.4738 - root_mean_squared_error: 92.6324 - mean_absolute_percentage_error: 99.1686 - val_loss: 7141.3716 - val_mean_absolute_error: 58.3772 - val_root_mean_squared_error: 84.5066 - val_mean_absol

1231/1231 [==============================] - 69s 56ms/step - loss: 370.4613 - mean_absolute_error: 13.7100 - root_mean_squared_error: 19.2474 - mean_absolute_percentage_error: 21.5460 - val_loss: 408.0188 - val_mean_absolute_error: 14.3957 - val_root_mean_squared_error: 20.1995 - val_mean_absolute_percentage_error: 25.0396
Epoch 50/100
1231/1231 [==============================] - 69s 56ms/step - loss: 369.1115 - mean_absolute_error: 13.6854 - root_mean_squared_error: 19.2123 - mean_absolute_percentage_error: 21.5575 - val_loss: 406.6240 - val_mean_absolute_error: 14.3858 - val_root_mean_squared_error: 20.1649 - val_mean_absolute_percentage_error: 25.6479
Epoch 51/100
1231/1231 [==============================] - 70s 57ms/step - loss: 366.2777 - mean_absolute_error: 13.6352 - root_mean_squared_error: 19.1384 - mean_absolute_percentage_error: 21.3825 - val_loss: 404.4504 - val_mean_absolute_error: 14.1948 - val_root_mean_squared_error: 20.1109 - val_mean_absolute_percentage_error: 22.2105

1231/1231 [==============================] - 234s 190ms/step - loss: 300.3817 - mean_absolute_error: 12.3962 - root_mean_squared_error: 17.3315 - mean_absolute_percentage_error: 19.2065 - val_loss: 408.7819 - val_mean_absolute_error: 14.0731 - val_root_mean_squared_error: 20.2184 - val_mean_absolute_percentage_error: 17.8993
Epoch 98/100
1231/1231 [==============================] - 194s 157ms/step - loss: 298.5356 - mean_absolute_error: 12.3534 - root_mean_squared_error: 17.2782 - mean_absolute_percentage_error: 19.0848 - val_loss: 412.0158 - val_mean_absolute_error: 14.1497 - val_root_mean_squared_error: 20.2982 - val_mean_absolute_percentage_error: 18.0122
Epoch 99/100
1231/1231 [==============================] - 232s 188ms/step - loss: 296.8274 - mean_absolute_error: 12.3198 - root_mean_squared_error: 17.2287 - mean_absolute_percentage_error: 19.0823 - val_loss: 409.3383 - val_mean_absolute_error: 14.1302 - val_root_mean_squared_error: 20.2321 - val_mean_absolute_percentage_error: 1

In [66]:
metrics5 = pd.DataFrame(model5.history.history)

In [67]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics5[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics5[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(400,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [68]:
model5.save("my_dir/10a-5")

INFO:tensorflow:Assets written to: my_dir/10a-5\assets


INFO:tensorflow:Assets written to: my_dir/10a-5\assets


In [69]:
model5 = keras.models.load_model("my_dir/10a-5") 

In [70]:
model5.predict((Wv5,Xv5))

array([[ 33.040283 ,  22.20416  ,  21.194714 ,  72.79177  ,  35.497444 ,
          7.6147714],
       [ 32.417057 ,  21.986277 ,  21.605656 ,  73.01562  ,  35.16079  ,
          8.074565 ],
       [ 31.410805 ,  19.921988 ,  19.21482  ,  69.98837  ,  34.01721  ,
          6.7701173],
       ...,
       [ 44.708282 ,  45.79256  ,  58.05465  , 112.04285  ,  51.388657 ,
         16.520914 ],
       [ 44.19042  ,  44.17982  ,  54.331802 , 109.39746  ,  49.720604 ,
         15.315227 ],
       [ 42.543262 ,  41.211    ,  52.431633 , 109.08073  ,  49.84595  ,
         14.769713 ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions5 = model5.predict((Wv5,Xv5))


MSE5 = mean_squared_error(yv5,test_predictions5)

MAE5 = mean_absolute_error(yv5,test_predictions5)

RMSE5 = np.sqrt(MSE5)

MAPE5 = mean_absolute_percentage_error(yv5,test_predictions5)*100

eval5 = [MSE5, MAE5, RMSE5, MAPE5]
print(eval5)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv5), np.asarray(test_predictions5))
GEH5 = count/GEH.size
print(GEH5)

# 15 mins ahead

In [73]:
def build_model_15(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt15.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt15.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_15w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_15 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_15w)
    
    
    
    # Deep Model
    # CNN 
    CNN_15 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_15 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_15)
    # LSTM          
    LSTM_15 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_15)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_15, LSTM_15], axis=1)

    # Attention
    Att_15 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_15 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_15.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_15

In [74]:
tuner15 = RandomSearch(
    build_model_15,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10a-15",
)

In [75]:
tuner15.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [76]:
tuner15.search((Wt15,Xt15), yt15, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop], 
            verbose=2)

Trial 10 Complete [03h 45m 16s]
val_loss: 460.9515380859375

Best val_loss So Far: 460.9515380859375
Total elapsed time: 1d 01h 23m 19s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [77]:
tuner15.results_summary(1)

Results summary
Results in C:\RunsOak\10a-15
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 09 summary
Hyperparameters:
units: 98
filters: 160
kernel_size: 2
learning_rate: 0.0001
Score: 460.9515380859375


### Best model

In [78]:
best_hp15 = tuner15.get_best_hyperparameters()[0]
model15 = tuner15.hypermodel.build(best_hp15)

In [79]:
scaler_filename = "Best_HP/10a-15"
joblib.dump(best_hp15, scaler_filename) 

['Best_HP/10a-15']

In [80]:
scaler_filename = "Best_HP/10a-15"
best_hp15 = joblib.load(scaler_filename) 

model15 = tuner15.hypermodel.build(best_hp15)

In [81]:
model15.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 160 4000        DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [82]:
model15.fit((Wt15,Xt15), yt15, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
1231/1231 [==============================] - 79s 60ms/step - loss: 22439.0371 - mean_absolute_error: 116.1856 - root_mean_squared_error: 149.7967 - mean_absolute_percentage_error: 93.4693 - val_loss: 16548.3125 - val_mean_absolute_error: 96.6499 - val_root_mean_squared_error: 128.6402 - val_mean_absolute_percentage_error: 115.3217
Epoch 2/100
1231/1231 [==============================] - 94s 76ms/step - loss: 11959.3105 - mean_absolute_error: 80.5243 - root_mean_squared_error: 109.3586 - mean_absolute_percentage_error: 116.0417 - val_loss: 9287.4434 - val_mean_absolute_error: 68.2868 - val_root_mean_squared_error: 96.3714 - val_mean_absolute_percentage_error: 105.3345
Epoch 3/100
1231/1231 [==============================] - 100s 81ms/step - loss: 6753.2681 - mean_absolute_error: 57.6403 - root_mean_squared_error: 82.1783 - mean_absolute_percentage_error: 97.5316 - val_loss: 5449.5596 - val_mean_absolute_error: 50.7514 - val_root_mean_squared_error: 73.8211 - val_mean_absolut

1231/1231 [==============================] - 338s 275ms/step - loss: 496.3419 - mean_absolute_error: 15.8268 - root_mean_squared_error: 22.2787 - mean_absolute_percentage_error: 25.1449 - val_loss: 492.6021 - val_mean_absolute_error: 15.5841 - val_root_mean_squared_error: 22.1946 - val_mean_absolute_percentage_error: 22.4842
Epoch 49/100
1231/1231 [==============================] - 316s 256ms/step - loss: 491.4695 - mean_absolute_error: 15.7503 - root_mean_squared_error: 22.1691 - mean_absolute_percentage_error: 24.9648 - val_loss: 488.8985 - val_mean_absolute_error: 15.5319 - val_root_mean_squared_error: 22.1111 - val_mean_absolute_percentage_error: 22.1464
Epoch 50/100
1231/1231 [==============================] - 212s 172ms/step - loss: 486.9839 - mean_absolute_error: 15.6647 - root_mean_squared_error: 22.0677 - mean_absolute_percentage_error: 24.7797 - val_loss: 491.0873 - val_mean_absolute_error: 15.5824 - val_root_mean_squared_error: 22.1605 - val_mean_absolute_percentage_error: 2

1231/1231 [==============================] - 83s 68ms/step - loss: 400.6391 - mean_absolute_error: 14.1726 - root_mean_squared_error: 20.0160 - mean_absolute_percentage_error: 22.1862 - val_loss: 462.7363 - val_mean_absolute_error: 15.1750 - val_root_mean_squared_error: 21.5113 - val_mean_absolute_percentage_error: 26.0217
Epoch 96/100
1231/1231 [==============================] - 83s 68ms/step - loss: 399.9839 - mean_absolute_error: 14.1590 - root_mean_squared_error: 19.9996 - mean_absolute_percentage_error: 22.2182 - val_loss: 461.4930 - val_mean_absolute_error: 15.0692 - val_root_mean_squared_error: 21.4824 - val_mean_absolute_percentage_error: 20.3348
Epoch 97/100
1231/1231 [==============================] - 84s 68ms/step - loss: 398.9299 - mean_absolute_error: 14.1392 - root_mean_squared_error: 19.9732 - mean_absolute_percentage_error: 22.0921 - val_loss: 462.7769 - val_mean_absolute_error: 15.0564 - val_root_mean_squared_error: 21.5123 - val_mean_absolute_percentage_error: 21.7755

In [83]:
metrics15 = pd.DataFrame(model15.history.history)

In [84]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics15[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics15[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [85]:
model15.save("my_dir/10a-15")

INFO:tensorflow:Assets written to: my_dir/10a-15\assets


INFO:tensorflow:Assets written to: my_dir/10a-15\assets


In [86]:
model15 = keras.models.load_model("my_dir/10a-15")

In [87]:
model15.predict((Wv15,Xv15))

array([[ 29.974127 ,  18.51719  ,  17.791315 ,  63.91576  ,  30.376251 ,
          5.434965 ],
       [ 28.995827 ,  17.23334  ,  17.396067 ,  63.593357 ,  30.254843 ,
          5.1000767],
       [ 28.973717 ,  16.789722 ,  16.582033 ,  62.805164 ,  29.925861 ,
          5.2324543],
       ...,
       [ 49.083817 ,  56.18886  ,  64.6376   , 125.795975 ,  59.11979  ,
         18.724407 ],
       [ 46.03093  ,  49.900856 ,  58.69641  , 119.98131  ,  56.653145 ,
         20.254866 ],
       [ 41.164642 ,  40.987244 ,  52.571426 , 111.411736 ,  51.861946 ,
         16.80966  ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions15 = model15.predict((Wv15,Xv15))


MSE15 = mean_squared_error(yv15,test_predictions15)

MAE15 = mean_absolute_error(yv15,test_predictions15)

RMSE15 = np.sqrt(MSE15)

MAPE15 = mean_absolute_percentage_error(yv15,test_predictions15)*100

eval15 = [MSE15, MAE15, RMSE15, MAPE15]
print(eval15)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv15), np.asarray(test_predictions15))
GEH15 = count/GEH.size
print(GEH15)

# 30 mins ahead

In [90]:
def build_model_30(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt30.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt30.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_30w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_30 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_30w)
    
    
    # Deep Model
    # CNN 
    CNN_30 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_30 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_30)
    # LSTM          
    LSTM_30 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_30)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_30, LSTM_30], axis=1)

    # Attention
    Att_30 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_30 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_30.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_30

In [91]:
tuner30 = RandomSearch(
    build_model_30,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10a-30",
)

In [92]:
tuner30.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [93]:
tuner30.search((Wt30,Xt30), yt30, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [00h 52m 03s]
val_loss: 644.2792053222656

Best val_loss So Far: 516.8050842285156
Total elapsed time: 1d 09h 52m 50s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [94]:
tuner30.results_summary(1)

Results summary
Results in C:\RunsOak\10a-30
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 00 summary
Hyperparameters:
units: 70
filters: 256
kernel_size: 2
learning_rate: 0.0001
Score: 516.8050842285156


### Best model

In [95]:
best_hp30 = tuner30.get_best_hyperparameters()[0]
model30 = tuner30.hypermodel.build(best_hp30)

In [96]:
scaler_filename = "Best_HP/10a-30"
joblib.dump(best_hp30, scaler_filename) 

['Best_HP/10a-30']

In [97]:
scaler_filename = "Best_HP/10a-30"
best_hp30 = joblib.load(scaler_filename) 

model30 = tuner30.hypermodel.build(best_hp30)

In [98]:
model30.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 256 6400        DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [99]:
model30.fit((Wt30,Xt30), yt30, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
1231/1231 [==============================] - 100s 76ms/step - loss: 24427.0449 - mean_absolute_error: 122.4001 - root_mean_squared_error: 156.2915 - mean_absolute_percentage_error: 88.7861 - val_loss: 19688.1602 - val_mean_absolute_error: 106.1674 - val_root_mean_squared_error: 140.3145 - val_mean_absolute_percentage_error: 100.2244
Epoch 2/100
1231/1231 [==============================] - 96s 78ms/step - loss: 15271.5566 - mean_absolute_error: 92.6185 - root_mean_squared_error: 123.5781 - mean_absolute_percentage_error: 112.9822 - val_loss: 12922.5898 - val_mean_absolute_error: 83.4688 - val_root_mean_squared_error: 113.6776 - val_mean_absolute_percentage_error: 118.1145
Epoch 3/100
1231/1231 [==============================] - 96s 78ms/step - loss: 9930.4336 - mean_absolute_error: 71.3274 - root_mean_squared_error: 99.6516 - mean_absolute_percentage_error: 103.5289 - val_loss: 8435.6133 - val_mean_absolute_error: 63.9514 - val_root_mean_squared_error: 91.8456 - val_mean_abs

1231/1231 [==============================] - 121s 98ms/step - loss: 740.8292 - mean_absolute_error: 18.9930 - root_mean_squared_error: 27.2182 - mean_absolute_percentage_error: 31.2722 - val_loss: 621.6444 - val_mean_absolute_error: 17.6806 - val_root_mean_squared_error: 24.9328 - val_mean_absolute_percentage_error: 28.2794
Epoch 26/100
1231/1231 [==============================] - 120s 97ms/step - loss: 727.5877 - mean_absolute_error: 18.8062 - root_mean_squared_error: 26.9738 - mean_absolute_percentage_error: 30.8389 - val_loss: 607.6261 - val_mean_absolute_error: 17.4598 - val_root_mean_squared_error: 24.6501 - val_mean_absolute_percentage_error: 28.9727
Epoch 27/100
1231/1231 [==============================] - 118s 96ms/step - loss: 716.8943 - mean_absolute_error: 18.6724 - root_mean_squared_error: 26.7749 - mean_absolute_percentage_error: 30.5933 - val_loss: 600.8387 - val_mean_absolute_error: 17.3859 - val_root_mean_squared_error: 24.5120 - val_mean_absolute_percentage_error: 28.9

1231/1231 [==============================] - 125s 101ms/step - loss: 484.8035 - mean_absolute_error: 15.5791 - root_mean_squared_error: 22.0183 - mean_absolute_percentage_error: 24.5123 - val_loss: 512.9542 - val_mean_absolute_error: 16.0047 - val_root_mean_squared_error: 22.6485 - val_mean_absolute_percentage_error: 26.3445
Epoch 73/100
1231/1231 [==============================] - 124s 101ms/step - loss: 483.1085 - mean_absolute_error: 15.5632 - root_mean_squared_error: 21.9797 - mean_absolute_percentage_error: 24.6401 - val_loss: 509.8329 - val_mean_absolute_error: 15.8205 - val_root_mean_squared_error: 22.5795 - val_mean_absolute_percentage_error: 22.8598
Epoch 74/100
1231/1231 [==============================] - 121s 98ms/step - loss: 480.9680 - mean_absolute_error: 15.5295 - root_mean_squared_error: 21.9310 - mean_absolute_percentage_error: 24.5515 - val_loss: 521.1891 - val_mean_absolute_error: 16.2858 - val_root_mean_squared_error: 22.8296 - val_mean_absolute_percentage_error: 28

1231/1231 [==============================] - 84s 68ms/step - loss: 445.8452 - mean_absolute_error: 14.9580 - root_mean_squared_error: 21.1150 - mean_absolute_percentage_error: 23.7787 - val_loss: 510.8584 - val_mean_absolute_error: 15.8945 - val_root_mean_squared_error: 22.6022 - val_mean_absolute_percentage_error: 24.4358
Epoch 97/100
1231/1231 [==============================] - 89s 73ms/step - loss: 445.2450 - mean_absolute_error: 14.9556 - root_mean_squared_error: 21.1008 - mean_absolute_percentage_error: 23.7930 - val_loss: 515.4670 - val_mean_absolute_error: 15.9430 - val_root_mean_squared_error: 22.7039 - val_mean_absolute_percentage_error: 21.2421
Epoch 98/100
1231/1231 [==============================] - 88s 72ms/step - loss: 443.7401 - mean_absolute_error: 14.9203 - root_mean_squared_error: 21.0651 - mean_absolute_percentage_error: 23.7336 - val_loss: 507.1872 - val_mean_absolute_error: 15.7597 - val_root_mean_squared_error: 22.5208 - val_mean_absolute_percentage_error: 21.8935

In [100]:
metrics30 = pd.DataFrame(model30.history.history)

In [101]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics30[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics30[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [102]:
model30.save("my_dir/10a-30")

INFO:tensorflow:Assets written to: my_dir/10a-30\assets


INFO:tensorflow:Assets written to: my_dir/10a-30\assets


In [103]:
model30 = keras.models.load_model("my_dir/10a-30")

In [104]:
model30.predict((Wv30,Xv30))

array([[ 29.635733 ,  17.682741 ,  14.332037 ,  61.468796 ,  29.605316 ,
          4.6176314],
       [ 29.951576 ,  17.957096 ,  14.0356455,  61.29625  ,  29.56679  ,
          4.606252 ],
       [ 29.29914  ,  17.14487  ,  14.910206 ,  61.87854  ,  29.828459 ,
          4.5007396],
       ...,
       [ 46.76372  ,  50.088966 ,  63.921627 , 122.95365  ,  54.33687  ,
         16.995619 ],
       [ 43.581284 ,  43.71205  ,  55.880238 , 112.49877  ,  49.427696 ,
         12.282961 ],
       [ 41.052296 ,  39.33162  ,  49.738503 , 103.01527  ,  46.070705 ,
         10.48734  ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions30 = model30.predict((Wv30,Xv30))


MSE30 = mean_squared_error(yv30,test_predictions30)

MAE30 = mean_absolute_error(yv30,test_predictions30)

RMSE30 = np.sqrt(MSE30)

MAPE30 = mean_absolute_percentage_error(yv30,test_predictions30)*100

eval30 = [MSE30, MAE30, RMSE30, MAPE30]
print(eval30)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv30), np.asarray(test_predictions30))
GEH30 = count/GEH.size
print(GEH30)

# 60 mins ahead

In [107]:
def build_model_60(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt60.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt60.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_60w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_60 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_60w)
    
    
    # Deep Model
    # CNN 
    CNN_60 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_60 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_60)
    # LSTM          
    LSTM_60 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_60)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_60, LSTM_60], axis=1)

    # Attention
    Att_60 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_60 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_60.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_60

In [108]:
tuner60 = RandomSearch(
    build_model_60,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10a-60",
)

In [109]:
tuner60.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [110]:
tuner60.search((Wt60,Xt60), yt60, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [04h 13m 45s]
val_loss: 773.4366455078125

Best val_loss So Far: 723.2718811035156
Total elapsed time: 1d 22h 35m 08s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [111]:
tuner60.results_summary(1)

Results summary
Results in C:\RunsOak\10a-60
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 05 summary
Hyperparameters:
units: 60
filters: 512
kernel_size: 3
learning_rate: 0.001
Score: 723.2718811035156


### Best model

In [112]:
best_hp60 = tuner60.get_best_hyperparameters()[0]
model60 = tuner60.hypermodel.build(best_hp60)

In [113]:
scaler_filename = "Best_HP/10a-60"
joblib.dump(best_hp60, scaler_filename) 

['Best_HP/10a-60']

In [114]:
scaler_filename = "Best_HP/10a-60"
best_hp60 = joblib.load(scaler_filename) 

model60 = tuner60.hypermodel.build(best_hp60)

In [115]:
model60.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 512 28160       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [116]:
model60.fit((Wt60,Xt60), yt60, validation_split = val_percent,
            epochs = 100, 
            verbose = 1
            )

Epoch 1/100
1231/1231 [==============================] - 81s 61ms/step - loss: 8725.9375 - mean_absolute_error: 63.0249 - root_mean_squared_error: 93.4127 - mean_absolute_percentage_error: 90.6548 - val_loss: 2227.2395 - val_mean_absolute_error: 33.3606 - val_root_mean_squared_error: 47.1936 - val_mean_absolute_percentage_error: 57.0772
Epoch 2/100
1231/1231 [==============================] - 89s 72ms/step - loss: 1768.5132 - mean_absolute_error: 30.1729 - root_mean_squared_error: 42.0537 - mean_absolute_percentage_error: 49.1426 - val_loss: 1357.0542 - val_mean_absolute_error: 26.9465 - val_root_mean_squared_error: 36.8382 - val_mean_absolute_percentage_error: 39.6666
Epoch 3/100
1231/1231 [==============================] - 102s 83ms/step - loss: 1431.6871 - mean_absolute_error: 26.9735 - root_mean_squared_error: 37.8376 - mean_absolute_percentage_error: 41.5212 - val_loss: 1050.5482 - val_mean_absolute_error: 23.3815 - val_root_mean_squared_error: 32.4122 - val_mean_absolute_percenta

1231/1231 [==============================] - 105s 85ms/step - loss: 948.5312 - mean_absolute_error: 20.6239 - root_mean_squared_error: 30.7982 - mean_absolute_percentage_error: 30.2202 - val_loss: 781.9026 - val_mean_absolute_error: 19.2767 - val_root_mean_squared_error: 27.9625 - val_mean_absolute_percentage_error: 26.2959
Epoch 45/100
1231/1231 [==============================] - 103s 83ms/step - loss: 948.4413 - mean_absolute_error: 20.6205 - root_mean_squared_error: 30.7968 - mean_absolute_percentage_error: 30.2677 - val_loss: 737.5743 - val_mean_absolute_error: 18.7996 - val_root_mean_squared_error: 27.1583 - val_mean_absolute_percentage_error: 27.3865
Epoch 46/100
1231/1231 [==============================] - 104s 85ms/step - loss: 946.7153 - mean_absolute_error: 20.6008 - root_mean_squared_error: 30.7687 - mean_absolute_percentage_error: 30.2820 - val_loss: 744.7487 - val_mean_absolute_error: 18.8747 - val_root_mean_squared_error: 27.2901 - val_mean_absolute_percentage_error: 25.6

1231/1231 [==============================] - 201s 163ms/step - loss: 901.3655 - mean_absolute_error: 20.1670 - root_mean_squared_error: 30.0227 - mean_absolute_percentage_error: 29.4792 - val_loss: 761.3184 - val_mean_absolute_error: 18.9695 - val_root_mean_squared_error: 27.5920 - val_mean_absolute_percentage_error: 24.8693
Epoch 87/100
1231/1231 [==============================] - 194s 157ms/step - loss: 901.1262 - mean_absolute_error: 20.1598 - root_mean_squared_error: 30.0188 - mean_absolute_percentage_error: 29.4638 - val_loss: 756.7475 - val_mean_absolute_error: 18.9338 - val_root_mean_squared_error: 27.5090 - val_mean_absolute_percentage_error: 25.4094
Epoch 88/100
1231/1231 [==============================] - 199s 162ms/step - loss: 899.2999 - mean_absolute_error: 20.1533 - root_mean_squared_error: 29.9883 - mean_absolute_percentage_error: 29.5090 - val_loss: 748.5214 - val_mean_absolute_error: 18.8131 - val_root_mean_squared_error: 27.3591 - val_mean_absolute_percentage_error: 2

In [117]:
metrics60 = pd.DataFrame(model60.history.history)

In [118]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics60[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics60[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [119]:
model60.save("my_dir/10a-60")

INFO:tensorflow:Assets written to: my_dir/10a-60\assets


INFO:tensorflow:Assets written to: my_dir/10a-60\assets


In [120]:
model60 = keras.models.load_model("my_dir/10a-60")

In [121]:
model60.predict((Wv60,Xv60))

array([[ 28.602158 ,  15.403277 ,  11.139361 ,  57.247555 ,  27.71213  ,
          5.306828 ],
       [ 28.52893  ,  15.352832 ,  10.651706 ,  55.521908 ,  27.65613  ,
          5.9499707],
       [ 28.150467 ,  15.013553 ,   8.452792 ,  52.37666  ,  26.582636 ,
          5.473912 ],
       ...,
       [ 45.127037 ,  45.970867 ,  71.73394  , 125.551956 ,  55.82381  ,
         26.44265  ],
       [ 42.520363 ,  41.628048 ,  63.992268 , 116.50794  ,  50.812943 ,
         21.118118 ],
       [ 39.78002  ,  37.92139  ,  62.586605 , 119.313095 ,  52.41268  ,
         20.479313 ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions60 = model60.predict((Wv60,Xv60))


MSE60 = mean_squared_error(yv60,test_predictions60)

MAE60 = mean_absolute_error(yv60,test_predictions60)

RMSE60 = np.sqrt(MSE60)

MAPE60 = mean_absolute_percentage_error(yv60,test_predictions60)*100

eval60 = [MSE60, MAE60, RMSE60, MAPE60]
print(eval60)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv60), np.asarray(test_predictions60))
GEH60 = count/GEH.size
print(GEH60)

# RESULTS

## General results

In [ ]:
l = ['MSE','MAE','RMSE','MAPE','Horizon']

In [ ]:
eval5.append(5)
eval15.append(15)
eval30.append(30)
eval60.append(60)

In [ ]:
df = pd.DataFrame(np.array([eval5,eval15,eval30,eval60]),columns=l)
df.set_index('Horizon')

## GEH

In [ ]:
n = [' 5 ',' 15 ',' 30 ',' 60 ']
df2 = pd.DataFrame(np.array([[GEH5, GEH15, GEH30, GEH60]]), columns=n)
print(df2)

## Save Loss

In [128]:
# metrics5.to_csv('Loss/8a-5.csv') 
# metrics15.to_csv('Loss/8a-15.csv') 
# metrics30.to_csv('Loss/8a-30.csv') 
# metrics60.to_csv('Loss/8a-60.csv') 